In [1]:
from sshtunnel import SSHTunnelForwarder
import paramiko
import pymysql
from paramiko import SSHClient
from os.path import expanduser
from forward import forward_tunnel
import subprocess as sbp
%matplotlib inline
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser
from matplotlib.pyplot import pie, axis, show
import pandas as pd
import difflib
import re
import distance

In [2]:
home = expanduser('~')
mypkey = paramiko.RSAKey.from_private_key_file("/home/udit/ipython/std/openssh-key")

sql_hostname =
sql_username =
sql_password =
sql_main_database = 
sql_port = 3306
ssh_host = 
ssh_user = 
ssh_pass='
ssh_port = 22
sql_ip = '1.1.1.1.1'

In [263]:
with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        #ssh_pkey=mypkey,
        ssh_password=ssh_pass,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
    conn = pymysql.connect(host='127.0.0.1', user=sql_username,
            passwd=sql_password, db=sql_main_database,
            port=tunnel.local_bind_port)
    query = '''select id as location_id,location_name,city_id,latitude,longitude,status from school_location_master'''
    school_location_master = pd.read_sql_query(query, conn)
    query = '''select id as city_id,city_name,state_id from school_city_master'''
    school_city_master = pd.read_sql_query(query, conn)
    query = '''select id as state_id,state_name,country_id from school_state_master'''
    school_state_master = pd.read_sql_query(query, conn)
    conn.close()
    
locations=pd.read_csv("IN.txt", header=None, sep="\t")
col_names=['country_code','postal_code','place_name','admin_name1','admin_code1','admin_name2','admin_code2','admin_code3','admin_code4','latitude','longitude','accuracy']
locations.columns=col_names
locations.admin_name1=locations.admin_name1.str.lower()
locations.place_name=locations.place_name.str.lower()
locations.loc[locations.admin_name1=='jammu & kashmir','admin_name1']='jammu and kashmir'
locations.loc[locations.admin_name1=='andaman & nicobar islands','admin_name1']='andaman and nicobar islands'
locations.loc[locations.admin_name1=='daman & diu','admin_name1']='daman and diu'
locations.loc[locations.admin_name1=='chattisgarh','admin_name1']='chhattisgarh'
locations.loc[locations.admin_name1=='dadra & nagar haveli','admin_name1']='dadra-and-nagar-haveli'
locations.loc[locations.admin_name1=='odisha','admin_name1']='orissa'
locations.loc[locations.admin_name1=='pondicherry','admin_name1']='puducherry'

In [337]:
def corect_location(location,list_names):
    match=difflib.get_close_matches(location, list_names)#locations.loc[locations.admin_name1=='west bengal','place_name'].values.tolist())
    if(len(match)>0):
        dis=distance.levenshtein(location,match[0])
        if(dis<4):
            return match[0]
        else:
            return location
    else:
        return location

In [265]:
df=pd.merge(school_location_master,school_city_master,on='city_id',how='left')
df=pd.merge(df,school_state_master,on='state_id',how='left')
df.city_name=df.city_name.str.lower()

In [266]:
df.to_csv("location.csv",index=False)
#STD_localities=pd.read_csv("city_localities.csv")

In [267]:
data=df
data['location_name']=data.location_name.str.lower()
data['location_name']=data.location_name.astype(str)

In [268]:
data=data.sort_values(['state_name','location_name'],ascending=False)

In [269]:
import re
data['location_name']=data['location_name'].apply(lambda x:re.sub('[^0-9a-zA-Z]+', ' ', x) )
data.state_name=data.state_name.str.lower()

In [270]:
data['location_name_2']=data.location_name.shift(1)
data['state_name_2']=data.state_name.shift(1)
data['location_id_2']=data.location_id.shift(1)
data['city_name_2']=data.city_name.shift(1)

In [271]:
data=data.dropna()

In [272]:
data['similarity']=data.apply(lambda x: difflib.SequenceMatcher(None, x['location_name'], x['location_name_2']).ratio(),axis=1)
data['lavisondistance']=data.apply(lambda x: distance.levenshtein(x['location_name'], x['location_name_2']),axis=1)

In [273]:
cols=['location_id','location_id_2','state_name','state_name_2','location_name','location_name_2','similarity','lavisondistance','city_name','city_name_2']
df=data[cols]

In [274]:
mostsimilar=df[(df.similarity>0.95)&(df.similarity<1)]

In [275]:
mostsimilar=mostsimilar[mostsimilar.city_name==mostsimilar.city_name_2]

In [280]:
mostsimilar=mostsimilar[mostsimilar.city_name==mostsimilar.city_name_2]
mostsimilar=mostsimilar[mostsimilar.state_name==mostsimilar.state_name_2]

In [282]:
#locations.loc[locations.admin_name1==states[21],'place_name'].values.tolist()

In [338]:
count=0
solution_df=pd.DataFrame()
for index,row in mostsimilar.iterrows():
    record_dict={}
    list_names=locations.loc[locations.admin_name1==row['state_name'],'place_name'].values.tolist()
    location_name_suggest=corect_location(row['location_name'],list_names)
    location_name_suggest_2=corect_location(row['location_name_2'],list_names)
    sugg_dist=distance.levenshtein(location_name_suggest,location_name_suggest_2)
    record_dict['location_id']=row['location_id']
    record_dict['location_id_2']=row['location_id_2']
    record_dict['location_name']=row['location_name']
    record_dict['location_name_2']=row['location_name_2']
    if(sugg_dist==0):
        record_dict['location_name_suggest']=location_name_suggest
        record_dict['location_name_suggest_2']=location_name_suggest_2
    else:
        record_dict['location_name_suggest']=None
        record_dict['location_name_suggest_2']=None
    solution_df=solution_df.append(pd.DataFrame(record_dict,index=[0]))

In [333]:
solution_df.to_csv("corrected_loactions.csv",index=False)

In [339]:
solution_df

,location_id,location_id_2,location_name,location_name_2,location_name_suggest,location_name_suggest_2
0,2414,17801.0,salt lake city,saltlake city,None,None
0,18368,2447.0,paschim barisa,paschim barisha,paschim barisha,paschim barisha
0,18255,17906.0,near bansdroni metro station,near bashdroni metro station,None,None
0,17888,18055.0,dum dum road,dumdum road,dumdum road,dumdum road
0,17688,2572.0,dakshineshwar,dakshineswar,dakshineswar,dakshineswar
0,17530,17246.0,via premnagar,via prem nagar,None,None
0,17233,17292.0,sahashtradhara road,sahastradhara road,sahastradhara road,sahastradhara road
0,17182,17265.0,general mahadeo singh road,general mahadev singh road,None,None
0,17534,17398.0,bhaniyawala,bhaniyawla,bhaniawala,bhaniawala
0,15984,16060.0,vikas nagar,vikash nagar,vikas nagar,vikas nagar


In [299]:
dis=distance.levenshtein(row['location_name_2'],difflib.get_close_matches(row['location_name'], list_names)[0])

In [243]:
row['location_name_2'],row['location_name']

('saltlake city', 'salt lake city')

In [21]:
from bs4 import BeautifulSoup

In [38]:
cities=pd.read_csv('cities.csv')

In [39]:
cities=cities.City.values.tolist()

In [42]:
cities_list={}
for c in cities:
    try:
        url="http://www.mapsofindia.com/"+c.lower()+"/localities/"
        print(url)
        page = urllib2.urlopen(url)
    except:
        continue
    #page = urllib2.urlopen(url)
    soup = BeautifulSoup(page)
    intrl_links=soup.find_all( class_='intrl_links')
    mumbailocality=[]
    for ilink in intrl_links:
        for name in ilink.find_all('li'):
            mumbailocality.append(name.string)
    cities_list[c]=mumbailocality

http://www.mapsofindia.com/mumbai/localities/
http://www.mapsofindia.com/delhi/localities/
http://www.mapsofindia.com/kolkata/localities/
http://www.mapsofindia.com/chennai/localities/
http://www.mapsofindia.com/hyderabad/localities/
http://www.mapsofindia.com/jaipur/localities/
http://www.mapsofindia.com/jabalpur/localities/
http://www.mapsofindia.com/agra/localities/
http://www.mapsofindia.com/varanasi/localities/
http://www.mapsofindia.com/lucknow/localities/
http://www.mapsofindia.com/kanpur/localities/
http://www.mapsofindia.com/allahabad/localities/
http://www.mapsofindia.com/ahmedabad/localities/
http://www.mapsofindia.com/chandigarh/localities/
http://www.mapsofindia.com/amritsar/localities/
http://www.mapsofindia.com/aizawl/localities/
http://www.mapsofindia.com/bhopal/localities/
http://www.mapsofindia.com/bhubaneshwar/localities/
http://www.mapsofindia.com/bhilai/localities/
http://www.mapsofindia.com/coimbatore/localities/
http://www.mapsofindia.com/darjeeling/localities/
h

In [121]:
STD_localities=pd.DataFrame.from_dict(cities_list,orient='index').T.dropna()
STD_localities=STD_localities.apply(lambda x:x.astype(str).str.lower())
STD_localities.columns=STD_localities.columns.str.lower()
STD_localities.to_csv("city_localities.csv",index=False)